In [ ]:
# 기사 목록

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import re
from datetime import datetime

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화



# Keywords to search
keywords = [
    "검색어",
]
tabs = ["NEWS", "EDITORIAL"]

# 검색 기간 설정
start_date = "2024/06/01"
end_date = "2024/08/31"
collected_urls = set()

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

def fetch_search_results(keyword, tab, start_date, end_date):
    """지정된 검색어와 탭, 날짜 범위로 한국일보에서 기사 스크래핑."""
    base_url = "https://search.hankookilbo.com/Search?Page="
    results = []
    page = 1  # 페이지 시작 번호 설정

    while True:
        print(f"{keyword} {tab} {page} 페이지 수집 중...")
        search_url = (
            f"{base_url}{page}&tab={tab}&sort=recent&searchText="
            f"{urllib.parse.quote(keyword)}&searchTypeSet=TITLE,CONTENTS&"
            f"startDate={start_date}%2000:00:00&endDate={end_date}%2023:59:59&filter=head"
        )
        driver.get(search_url)

        try:
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.CLASS_NAME, "board-list"))
            )
        except Exception as e:
            print(f"페이지 로딩 중 오류 발생: {e}")
            break

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        articles = soup.find_all("li")

        if not articles:  # 더 이상 기사가 없을 때 종료
            break

        new_articles_found = False  # 새로운 기사가 발견되었는지 확인하는 플래그

        for article in articles:
            try:
                div_tag = article.find("div", class_="inn")
                if div_tag:
                    title_tag = div_tag.find("h3", class_="board-list h3 pc_only")
                    title = title_tag.find("a").text.strip() if title_tag else ""
                    url = title_tag.find("a")["href"] if title_tag else ""

                    # 중복 URL 체크
                    if url in collected_urls:
                        continue  # 중복된 URL은 무시
                    collected_urls.add(url)  # 새로운 URL 저장
                    new_articles_found = True  # 새로운 기사가 발견됨

                    # 날짜 추출
                    date_element = div_tag.find("p", class_="date pc_only")
                    date = date_element.text.split(' ')[0] if date_element else ""

                    # 저자 추출
                    writer = ""
                    writer_element = date_element.find("em") if date_element else None
                    if writer_element and writer_element.next_sibling:
                        writer = writer_element.next_sibling.strip()
                        if re.match(r"[^@]+@[^@]+\.[^@]+", writer):
                            writer = ""  # 이메일 형식이면 무시

                    section = "사설/칼럼" if tab == "EDITORIAL" else "뉴스"

                    # 결과 리스트에 추가
                    results.append({
                        'media': '한국일보',
                        'title': title,
                        'date': date,
                        'section': section,
                        'article_type': '',
                        'writer': writer,
                        'page': '',
                        'url': url,
                        'content': ''  # 컨텐츠 정보 없음
                    })
            except Exception as e:
                print(f"기사 정보 추출 중 오류 발생: {e}")

        if not new_articles_found:  # 새로운 기사가 없으면 종료
            break

        page += 1  # 다음 페이지로 이동

    return results


def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)

all_results = []
try:
    for keyword in keywords:
        for tab in tabs:
            all_results.extend(fetch_search_results(keyword, tab, start_date, end_date))

    df = pd.DataFrame(all_results)

    # 기사 수 출력
    print(f"Total number of articles: {len(df)}")

    # 문자열을 datetime 객체로 변환
    start_date_obj = datetime.strptime(start_date, '%Y/%m/%d')
    end_date_obj = datetime.strptime(end_date, '%Y/%m/%d')

    # 날짜를 'YYYY-MM-DD' 형식으로 변환
    start_date_str = start_date_obj.strftime('%Y-%m-%d')
    end_date_str = end_date_obj.strftime('%Y-%m-%d')

    # 중복 제거된 데이터 저장
    save_to_excel(df, f'1한국일보 {start_date_str}~{end_date_str}.xlsx')

finally:
    # 드라이버 종료
    driver.quit()


In [ ]:
# 본문 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문과 서브 타이틀을 스크랩하는 함수 정의
def get_content_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        article_body = soup.find('div', class_='end-body').find('div', class_='innerwrap').find('div', class_='col-main') if soup.find('div', class_='end-body') else None

        # 서브 타이틀 추출 (col-main 내부에서 div.sub-tit-ll)
        subtitle = ""
        if article_body:
            subtitle_tag = article_body.find('h2', class_='sub-tit-ll')
            if subtitle_tag:
                subtitle = subtitle_tag.get_text(separator='\n', strip=True)
                subtitle_tag.decompose()  # 본문에서 서브 타이틀 제거

            # 'editor-img-box' 클래스를 가진 모든 div 태그를 제거합니다.
            for caption in article_body.find_all('div', class_='editor-img-box'):
                caption.decompose()

            # 'figcaption' 태그는 태그 자체를 삭제
            for figcaption_tag in article_body.find_all('figcaption'):
                figcaption_tag.decompose()

            # 'report-banner-pnl' 클래스를 가진 section 태그 제거
            for section_tag in article_body.find_all('section', class_='report-banner-pnl'):
                section_tag.decompose()

            # btn-area 태그 이후의 태그들도 제거
            btn_area = article_body.find('div', class_='btn-area reaction-btns')
            if btn_area:
                next_sibling = btn_area.find_next_sibling()
                while next_sibling:
                    next_sibling.decompose()
                    next_sibling = btn_area.find_next_sibling()
                btn_area.decompose()  # btn_area 자체도 제거

            content = article_body.get_text(separator='\n', strip=True)
        else:
            content = ""

        return content, subtitle

    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문과 서브 타이틀 내용을 스크랩하여 content와 sub-title 컬럼에 저장합니다.
for index, row in df.iterrows():
    # 이미 content가 있는 경우 해당 URL 스킵
    if pd.notna(row.get('content')) and row['content'].strip():
        print(f"Skipping {index + 1}/{total_articles}: {row['url']} (이미 컨텐츠가 존재)")
        continue

    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_from_url(row['url'])

    if content and content.startswith('='):
        content = "'" + content

    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle  # 서브 타이틀 추가

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")
